In [10]:
import pandas as pd
from preprocess.setup import Preprocess
from transformers import AutoTokenizer

In [31]:
text = "Pasienten har også opplevd økt tungpust og hoste de siste månedene, noe som har begrenset aktivitetsnivået hans. Han vil bli behandlet med EKG snarest. Pasienten har også opplevd økt tungpust og hoste de siste månedene, noe som har begrenset aktivitetsnivået hans. Han vil bli behandlet med EKG snarest."

In [32]:
checkpoint = "ltg/norbert3-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
preprocess = Preprocess(tokenizer, 100, 0.8)

In [33]:
output = preprocess.run(text)

In [46]:
output

[Encoding(num_tokens=100, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [35]:
mer_output = [['O', 'O', 'O', 'O', 'O', 'O', 'B-SYMPTOM', 'I-SYMPTOM', 'I-SYMPTOM', 'O', 'B-SYMPTOM', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EVENT', 'I-EVENT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SYMPTOM', 'I-SYMPTOM', 'I-SYMPTOM', 'O', 'B-SYMPTOM', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]

In [ ]:
def get_non_o_intervals(lst):
    intervals = []
    start = None
    
    for i, value in enumerate(lst):
        if value != 'O' and (start is None or not value.startswith('B')):
            if start is None:  # Starting a new interval
                start = i
        else:
            if start is not None:  # Closing an existing interval
                intervals.append((start, i))
                start = None
    
    # If the last element is part of an interval
    if start is not None:
        intervals.append((start, len(lst) - 1))
    
    return intervals

result = get_non_o_intervals(mer_output[0])

print(result)

entities = []
start = 0
offset = 0
for int in result:
    entity = preprocess.decode(output[0].ids[int[0]:int[1]]).strip()
    found = -1
    while found == -1 and len(output[0].ids[0:int[1]+offset]) != len(output[0].ids):
        offset += 1
        context = preprocess.decode(output[0].ids[start:int[1]+offset])
        found = context.find('.')
        if -1 < found < context.find(entity):
            start = start + 1
            offset = offset - 1
            found = -1
    offset = 0
    context = context.replace('[CLS]', '').replace('[SEP]', '').replace('[PAD]', '')
    entities.append((entity, context))
    
entities

[(6, 9), (10, 11), (27, 29), (41, 44), (45, 46)]


[('tungpust',
  ' Pasienten har også opplevd økt tungpust og hoste de siste månedene, noe som har begrenset aktivitetsnivået hans.'),
 ('hoste',
  ' Pasienten har også opplevd økt tungpust og hoste de siste månedene, noe som har begrenset aktivitetsnivået hans.'),
 ('EKG', ' Han vil bli behandlet med EKG snarest.'),
 ('de siste månedene',
  ' Pasienten har også opplevd økt tungpust og hoste de siste månedene, noe som har begrenset aktivitetsnivået hans.'),
 ('noe',
  ' Pasienten har også opplevd økt tungpust og hoste de siste månedene, noe som har begrenset aktivitetsnivået hans.')]

In [45]:
inputs = []

for i, e_i in enumerate(entities):
    for j, e_j in enumerate(entities):
        if i == j:
            continue
        inputs.append(f"{e_i[0]}: {e_i[1]} [SEP] {e_j[0]}: {e_j[1]}")
        
inputs

['tungpust:  Pasienten har også opplevd økt tungpust og hoste de siste månedene, noe som har begrenset aktivitetsnivået hans. [SEP] hoste:  Pasienten har også opplevd økt tungpust og hoste de siste månedene, noe som har begrenset aktivitetsnivået hans.',
 'tungpust:  Pasienten har også opplevd økt tungpust og hoste de siste månedene, noe som har begrenset aktivitetsnivået hans. [SEP] EKG:  Han vil bli behandlet med EKG snarest.',
 'tungpust:  Pasienten har også opplevd økt tungpust og hoste de siste månedene, noe som har begrenset aktivitetsnivået hans. [SEP] de siste månedene:  Pasienten har også opplevd økt tungpust og hoste de siste månedene, noe som har begrenset aktivitetsnivået hans.',
 'tungpust:  Pasienten har også opplevd økt tungpust og hoste de siste månedene, noe som har begrenset aktivitetsnivået hans. [SEP] noe:  Pasienten har også opplevd økt tungpust og hoste de siste månedene, noe som har begrenset aktivitetsnivået hans.',
 'hoste:  Pasienten har også opplevd økt tungp